<a href="https://colab.research.google.com/github/mjavadzadeh11/Crawler/blob/master/Sentimental%20analysis%20using%20Bitcoin%20historical%20datas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install GoogleNews
!pip install newspaper3k
#!pip install vaderSentiment;

In [ ]:
!pip install googleanalytics


In [ ]:
import pandas as pd
import datetime as dt
import json
import matplotlib.pyplot as plt
from GoogleNews import GoogleNews
from newspaper import Article
import requests
import nltk
nltk.download('punkt')
from textblob import TextBlob
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [153]:
googlenews = GoogleNews(period='30d')
#googlenews = GoogleNews(start='11/04/2021',end='12/04/2021')
googlenews.search('Iran')
googlenews.get_page(30)
Result=googlenews.results()

In [ ]:
def get_news(keyword , period, NumOfPages):
#  ''' keyword is something that will be searched in google , period is the period of search
#  and NumOfPages is how many pages def should search'''
  googlenews = GoogleNews(period=period)
  googlenews.search(keyword)
  for i in range(2,NumOfPages):
    googlenews.getpage(i)
    result=googlenews.result()
    df=pd.DataFrame(result)
  return df

df=get_news('Crypto', '50d',50)

In [165]:
#df.drop(columns=['datetime'], inplace=True)
df.drop(columns=['img'], inplace=True)

In [ ]:
#filtering by media
filt = df['media']== 'Al Jazeera'
df[filt]

In [ ]:
#grouping by media
media_gp= df.groupby(['media'])
media_gp ['link'].value_counts() 

In [166]:

for index in range(df.shape[0]):
  try:
    article=Article(df.loc[index,'link'])
    article.download()
    article.html
    article.parse()
    article.nlp()
    
    
    df.loc[index,'Context'] = article.text
    df.loc[index, 'Summary'] = article.summary
    df.loc[index, 'Keywords'] = ' '.join(article.keywords)
  except:
    df.loc[index,'Context'] = 'Not able to recieve this news'
    df.loc[index, 'Summary'] = 'Could not summarize the context'
    df.loc[index, 'Keywords'] = "No keywords founded"
    

#df['Context'] = news_context

In [167]:
df.loc[:,'datetime']=df.loc[:,'datetime'].apply(lambda x: x.date())

In [168]:

for i in range(df.shape[0]):
  if type(df.loc[i,'datetime']) == pd._libs.tslibs.nattype.NaTType:
    if df.loc[i,'date'].split(' ')[1]=='mins':
      df.loc[i,'datetime'] = dt.date.today()
    elif df.loc[i,'date'].split(' ')[1]=='day':
      df.loc[i,'datetime'] = dt.date.today() - dt.timedelta(1)
    elif df.loc[i,'date'].split(' ')[1]=='days':
      df.loc[i,'datetime'] = dt.date.today() - dt.timedelta(df.loc[i,'date'].split(' ')[0])


In [169]:
df.set_index('datetime', inplace=True)

In [ ]:
df.head()

In [170]:
def get_polarity(text):
  return TextBlob(text).sentiment.polarity

In [171]:
title_polarity=df.loc[:,'title'].apply(get_polarity)
desc_polarity=df.loc[:,'desc'].apply(get_polarity)
df['Polarity']=(title_polarity+desc_polarity)/2

In [ ]:
n_news=df.groupby('datetime').count()['Polarity']

In [ ]:
polarity_avg = df.groupby(['datetime']).mean()
polarity_avg

In [172]:
#getting candel datas from binance
def get_binance_datas (symbol, interval , startTime , endTime):
    ''' symbol="ETHUSDT or BTCUSDT , ..." , interval="1h,4h,8h,12h,1d", times = dt.datetime(2020,1,1) '''
    
    url = "https://api.binance.com/api/v3/klines"
    startTime= str(int(startTime.timestamp()*1000))
    endTime= str(int(endTime.timestamp()*1000))
    limit ="1000"
    req_p = {"symbol" : symbol , "interval": interval , 'startTime':startTime , 'endTime':endTime , 'limit':limit}
    response = pd.DataFrame(json.loads(requests.get(url , params = req_p).text))
    if len(response.index)==0 :
        return None
    response=response.iloc[:,0:6]
    response.columns=['Datetime' , 'Open' , 'High' , 'Low' , 'Close' , 'Volume' ]
    response['Datetime'] = [dt.datetime.fromtimestamp(x/1000) for x in response['Datetime']]
    response.set_index(['Datetime'] , inplace=True)
    return response
    

In [177]:
df1=get_binance_datas('BTCUSDT' , '1d' , dt.datetime (2021,12,1) , dt.datetime(2021,12,7))

In [173]:
months = [dt.datetime(2021,i,1) for i in range(1,11)]
#months.append(dt.datetime(2021,1,1))
Symbol = 'BTCUSDT'
TimeFrame = '1d' #1m , 3m , 5m , 15m , 30m, 1h, 2h, 4h, 6h, 8h, 12h , 1d, 3d, 1w , 1M

datalist = [get_binance_datas (Symbol , TimeFrame , months[i] , months[i+1]-dt.timedelta(0,1)) for i in range(0,len(months)-1)]
df1 = pd.concat(datalist)
Name = Symbol + " " + TimeFrame

In [178]:

df1['Mean'] = (df1.loc[:,'Open'].apply(float) + df1.loc[:,'Close'].apply(float)) / 2

In [188]:
num=df.groupby(['datetime']).count()['title']


In [ ]:
#plot 1 BTC daily chart
plt.figure(figsize=(12.33 , 20))
plt.subplot(3, 1, 1)
plt.plot(df1.index ,df1.Mean)
plt.title('BTC chart ')
plt.xlabel('Date')
plt.ylabel('Price(usdt)')

# plot 2 news impact

plt.figure(figsize=(12.33 , 20))
plt.subplot(3, 1, 2)
plt.plot(polarity_avg.index ,polarity_avg)
plt.title('polarity of news based on date')
plt.xlabel('Date')
plt.ylabel('Polarity')

#plot 3
plt.figure(figsize=(12.33,20))
plt.subplot(3, 1, 3)
plt.plot(num.index , num)
plt.title('Number of news base on date')
plt.xlabel('Date')
plt.ylabel('Num of News')
plt.show()